In [20]:
from utils import load_deepfake_dataset
from dataloader.Loader import Loader
import os
import pandas as pd

class VoxCelebLoader(Loader):
    def read_data(self, directory):
        base = f"{os.getcwd()}/../data/VoxCeleb"
        data_list = []
        for set in directory:
            split = set['split']

            wav_dir = f"{base}/{set['name']}/{set['split']}/wav"
            aac_dir = f"{base}/{set['name']}/{set['split']}/aac"
            if os.path.exists(wav_dir):
                dir = wav_dir
            elif os.path.exists(aac_dir):
                dir = aac_dir

            for root, dirs, files in os.walk(dir):
                for file in files:
                    if not file.endswith(".wav") and not file.endswith(".m4a"):
                        continue
                    utterance = os.path.splitext(
                        os.path.basename(os.path.join(root, file)))[0]
                    id = os.path.join(root, file).split("/")[-3]
                    speaker_id = int(id[2:])
                    data_list.append({
                        'filename': os.path.normpath(os.path.join(root, file)),
                        'utterance': utterance,
                        'speaker': speaker_id,
                        'method_type': 'bonafide',
                        'method_name': 'bonafide',
                        'vocoder': 'bonafide',
                        'is_genuine': 1
                    })
        self.data_list = pd.DataFrame(data_list, columns=[
            'filename', 'utterance', 'speaker', 'method_type', 'method_name', 'vocoder', 'is_genuine',])

def create_csv(name, split):
    ds = [{"name": name, "split": split}]
    loader = VoxCelebLoader(ds, lambda x: x)
    loader.data_list.to_csv(f"../data/VoxCeleb/{name}/{split}/metadata.csv", index=False)

In [21]:
create_csv("VoxCeleb1", "test")

In [22]:
create_csv("VoxCeleb2", "test")

In [24]:
create_csv("VoxCeleb1", "dev")

In [23]:
create_csv("VoxCeleb2", "dev")

In [2]:
import pandas as pd

def rep(file_path):
    # Step 1: Read the CSV
    data = pd.read_csv(file_path)

    # Step 2: Create function to construct the new utterance id
    def construct_utterance_id(filename):
        parts = filename.split('/')
        return f"{parts[-3]}_{parts[-2]}_{parts[-1].split('.')[0]}"

    # Step 3: Apply function to filename and save to utterance
    data['utterance'] = data['filename'].apply(construct_utterance_id)

    data.head()
    # Step 4: Save the file
    data.to_csv(file_path, index=False)

rep('../data/VoxCeleb/VoxCeleb1/dev/metadata.csv')
rep('../data/VoxCeleb/VoxCeleb1/test/metadata.csv')
rep('../data/VoxCeleb/VoxCeleb2/dev/metadata.csv')
rep('../data/VoxCeleb/VoxCeleb2/test/metadata.csv')